# Perceptron Algorithm Definition

In this module, I will write and implement the perceptron algorithm, which is a supervised learning algorithm for binary classifiers. It is one of the simplest types of artificial neural networks and operates by finding a linear decision boundary that separates data points into two classes. The algorithm functions as follows:

## Initialization

The algorithm starts by initializing the weight vector and bias term to zero (or small random values). These parameters define the separating hyperplane.

## Prediction

For each input sample, the algorithm computes the weighted sum of the input features and the bias. It then applies an activation function — specifically, the unit step function — to determine the predicted class label:

\[
\text{Prediction} =
\begin{cases}
1 & \text{if } (w \cdot x + b) \geq 0 \\
0 & \text{otherwise}
\end{cases}
\]

## Update Rule

If the prediction does not match the true label, the model updates its weights and bias to reduce future errors. The updates are given by:

\[
w \leftarrow w + \eta (y_i - \hat{y}_i) x_i
\]
\[
b \leftarrow b + \eta (y_i - \hat{y}_i)
\]

where:
- \( w \) is the weight vector,
- \( b \) is the bias,
- \( \eta \) is the learning rate,
- \( y_i \) is the true label,
- \( \hat{y}_i \) is the predicted label,
- \( x_i \) is the input vector.

## Iteration

The algorithm repeats the prediction and update steps over the training data for a fixed number of iterations or until the model converges (i.e., no further updates are needed).

The Perceptron algorithm is guaranteed to converge to a solution if the data is linearly separable. If the data is not linearly separable, the algorithm will continue updating without finding a perfect solution.
